In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.nn.pool import global_mean_pool
from torch.utils.data import ConcatDataset, DataLoader, random_split, Sampler

In [2]:
from dataset import  CVFConfigForGCNWSuccLSTMDataset

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
# dataset_s_n7 = CVFConfigForGCNWSuccLSTMDataset(
#     device,
#     "star_graph_n7_config_rank_dataset.csv",
#     "star_graph_n7_edge_index.json",
# )

# dataset_rr_n7 = CVFConfigForGCNWSuccLSTMDataset(
#     device,
#     "graph_random_regular_graph_n7_d4_config_rank_dataset.csv",
#     "graph_random_regular_graph_n7_d4_edge_index.json",
# )

# dataset_plc_n7 = CVFConfigForGCNWSuccLSTMDataset(
#     device,
#     "graph_powerlaw_cluster_graph_n7_config_rank_dataset.csv",
#     "graph_powerlaw_cluster_graph_n7_edge_index.json",
# )


dataset_implict_n7 = CVFConfigForGCNWSuccLSTMDataset(
    device,
    "implicit_graph_n7_config_rank_dataset.csv",
    "implicit_graph_n7_edge_index.json",
    "dijkstra",
)



batch_size = 64

dataset_coll = [
    dataset_implict_n7,
]

In [5]:
train_sizes = [int(0.95 * len(ds)) for ds in dataset_coll]
test_sizes = [len(ds) - trs for ds, trs in zip(dataset_coll, train_sizes)]

train_test_datasets = [
    random_split(ds, [tr_s, ts])
    for ds, tr_s, ts in zip(dataset_coll, train_sizes, test_sizes)
]

train_datasets = [ds[0] for ds in train_test_datasets]
test_datasets = [ds[1] for ds in train_test_datasets]

In [6]:
datasets = ConcatDataset(train_datasets)
print(f"Train Dataset size: {len(datasets):,}")

Train Dataset size: 2,077


In [7]:
class CustomBatchSampler(Sampler):
    def __init__(self, datasets: ConcatDataset, batch_size: int):
        self.datasets = datasets
        self.batch_size = batch_size

    def __iter__(self):
        last_accessed = [0] + self.datasets.cumulative_sizes[:]
        end_loop = [False for _ in range(len(self.datasets.datasets))]

        while not all(end_loop):
            for turn in range(len(self.datasets.datasets)):
                if end_loop[turn]:
                    continue

                batch_size = self.batch_size
                if (
                    last_accessed[turn] + batch_size
                    >= self.datasets.cumulative_sizes[turn]
                ):
                    batch_size = (
                        self.datasets.cumulative_sizes[turn] - last_accessed[turn]
                    )
                    end_loop[turn] = True

                yield list(range(last_accessed[turn], last_accessed[turn] + batch_size))

                last_accessed[turn] += batch_size

In [8]:
batch_sampler = CustomBatchSampler(datasets, batch_size=batch_size)
dataloader = DataLoader(datasets, batch_sampler=batch_sampler)

In [9]:
class SimpleLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.lstm = nn.GRU(input_size, hidden_size, batch_first=True)
        self.norm = nn.LayerNorm(hidden_size)
        self.h2o = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        output, _ = self.lstm(x)
        output = self.norm(output)
        output = self.h2o(output)
        output = torch.relu(output)
        output = global_mean_pool(output, torch.zeros(output.size(1)).to(device).long())
        return output

    def fit(self, epochs):
        criterion = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=0.0001)
        for epoch in range(1, epochs + 1):
            self.train()
            total_loss = 0
            count = 0
            for batch in dataloader:
                x = batch[0]
                y = batch[1]
                y = y.unsqueeze(-1)
                out = self(x[0])
                optimizer.zero_grad()
                loss = criterion(out, y)
                total_loss += loss
                count += 1
                loss.backward()
                optimizer.step()

            print(
                "Training set | Epoch",
                epoch,
                "| MSE Loss:",
                round((total_loss / count).item(), 4),
            )

In [10]:
D = 7
H = 16

model = SimpleLSTM(D, H, 1).to(device)
print(model)
print()
print("Total parameters:", f"{sum(p.numel() for p in model.parameters()):,}")
print()
model.fit(epochs=200)

SimpleLSTM(
  (lstm): GRU(7, 16, batch_first=True)
  (norm): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
  (h2o): Linear(in_features=16, out_features=1, bias=True)
)

Total parameters: 1,249

Training set | Epoch 1 | MSE Loss: 435.8536
Training set | Epoch 2 | MSE Loss: 160.35
Training set | Epoch 3 | MSE Loss: 118.497
Training set | Epoch 4 | MSE Loss: 115.2849
Training set | Epoch 5 | MSE Loss: 98.7019
Training set | Epoch 6 | MSE Loss: 87.402
Training set | Epoch 7 | MSE Loss: 82.9767
Training set | Epoch 8 | MSE Loss: 76.1981
Training set | Epoch 9 | MSE Loss: 66.1714
Training set | Epoch 10 | MSE Loss: 59.0823
Training set | Epoch 11 | MSE Loss: 54.8881
Training set | Epoch 12 | MSE Loss: 53.8995
Training set | Epoch 13 | MSE Loss: 54.3577
Training set | Epoch 14 | MSE Loss: 50.0994
Training set | Epoch 15 | MSE Loss: 48.5437
Training set | Epoch 16 | MSE Loss: 47.0107
Training set | Epoch 17 | MSE Loss: 45.4873
Training set | Epoch 18 | MSE Loss: 43.8814
Training set | E

In [11]:
import csv
import datetime

f = open(f"test_results/test_result_w_succ_diff_nodes_gru_{datetime.datetime.now().strftime("%Y_%m_%d_%H_%M")}.csv", "w", newline="")
csv_writer = csv.writer(f)
csv_writer.writerow(["Dataset", "Actual", "Predicted"])

criterion = torch.nn.MSELoss()

model.eval()

with torch.no_grad():
    test_concat_datasets = ConcatDataset(test_datasets)
    test_batch_sampler = CustomBatchSampler(test_concat_datasets, batch_size=10240)
    test_dataloader = DataLoader(test_concat_datasets, batch_sampler=test_batch_sampler)

    total_loss = 0
    total_matched = 0
    count = 0
    for batch in test_dataloader:
        x = batch[0]
        y = batch[1]
        y = y.unsqueeze(-1)
        out = model(x[0])
        csv_writer.writerows(
            (i, j.item(), k.item())
            for (i, j, k) in zip(
                 batch[0][1], y.detach().cpu().numpy(), out.detach().cpu().numpy()
            )
        )
        loss = criterion(out, y)
        total_loss += loss
        out = torch.round(out)
        matched = (out == y).sum().item()
        total_matched += matched
        count += 1

    print(
        "Test set",
        "| MSE loss:",
        round((total_loss / count).item(), 4),
        "| Total matched",
        total_matched,
        "out of",
        len(test_concat_datasets),
        f"(Accuracy: {round(total_matched/len(test_concat_datasets) * 100, 2)}%)",
    )

f.close()

Test set | MSE loss: 32.3723 | Total matched 8 out of 110 (Accuracy: 7.27%)
